In [1]:
import xml.etree.ElementTree as xml
from glob import glob
import os
from tqdm import tqdm

In [2]:
raw_data_path = r"../data/raw/*"
processed_data_path = '../data/processed'

In [3]:
if not os.path.isdir(processed_data_path):
    os.mkdir(processed_data_path)

In [4]:
path = glob(raw_data_path)
# path[0]
# print(path[0].split('/')[-1][:-4])

## Run the script to generate `.txt` files from `.xml` documents

### TODO:

- replace URLs with an appropiate `<TAG>` 
- there are some portions of text like this N #176;1, it needs to be changed

In [5]:
for p in tqdm(path):
    subject = p.split('/')[-1][:-4] + ".txt"
    # print(subject)
    subject_path = os.path.join(processed_data_path, subject)
    if os.path.isfile(subject_path):
        continue
    root = xml.parse(p).getroot()
    for text in root.findall("WRITING/TEXT"):
        with open(subject_path, 'a') as f:
            if text.text is not None:
                f.write(text.text)
            else:
                f.write(" ")
            f.write("\n")

100%|██████████| 2348/2348 [00:00<00:00, 6173.94it/s]


In [34]:
# path_labels = '../data/risk_golden_truth.txt'
# labels = {}
# with open(path_labels, 'r') as f:
#     for line in f:
#         subject, label = line.split()
#         labels[subject] = int(label)
# labels


In [24]:
from torch.utils.data import DataLoader, Dataset

In [45]:
class TXTDataset(Dataset):
    def __init__(self, path_to_data="../data"):
        self.labels = []
        self.path_to_data = path_to_data
        self.ext = ".txt"
        self.path_to_processed= os.path.join(self.path_to_data, "processed")
        path_labels = os.path.join(path_to_data, "risk_golden_truth.txt")
        with open(path_labels, 'r') as f:
            for line in f:
                subject, label = line.split()
                self.labels.append((subject, label))
    
    def __getitem__(self, idx):
        subject, label = self.labels[idx]
        subject_path = os.path.join(self.path_to_processed, subject) + self.ext
        with open(subject_path, 'r') as f:
            text = " ".join(f.readlines())

        return (text, label)

    def __len__(self):
        return len(self.labels)




In [46]:
ds = TXTDataset()

## 1 user X texts -> from each text create a sample

In [3]:
raw_data_path = r"../data/raw/*"
split_data_path = '../data/split'
if not os.path.isdir(split_data_path):
    os.mkdir(split_data_path)

In [4]:
path_labels = os.path.join('../data/', "risk_golden_truth.txt")
users = []
with open(path_labels, 'r') as f:
    for line in f:
        subject, label = line.split()
        users.append((subject, label))

In [5]:
ground_truth_subtexts = os.path.join('../data/', 'risk_golden_truth_split.txt')
for subject, label in tqdm(users):
    user_file = os.path.join('../data', 'raw', subject+'.xml')

    root = xml.parse(user_file).getroot()
    for i, text in enumerate(root.findall("WRITING/TEXT")):
        subject_subtext_path = os.path.join(split_data_path, f"{subject}_{i}.txt")
        # xml to txt
        if text.text is None:
            continue
        with open(subject_subtext_path, 'w') as f:
            f.write(text.text)
        # generate labels
        with open(ground_truth_subtexts, 'a') as f:
            f.write(f'{subject}_{i} {label}\n')


100%|██████████| 2348/2348 [01:42<00:00, 22.88it/s]


In [6]:
path = glob(raw_data_path)

In [ ]:
for p in tqdm(path):
    subject = p.split('/')[-1][:-4] + ".txt"
    # print(subject)
    subject_path = os.path.join(processed_data_path, subject)
    if os.path.isfile(subject_path):
        continue
    root = xml.parse(p).getroot()
    for text in root.findall("WRITING/TEXT"):
        with open(subject_path, 'a') as f:
            if text.text is not None:
                f.write(text.text)
            else:
                f.write(" ")
            f.write("\n")